# Load libraries

In [ ]:
%load_ext autoreload
%autoreload 2
from images import images_path
from utils.simulator import FdaSimulator
import matplotlib.pyplot as plt
import numpy as np
import os

# Generate the data

In [ ]:
fda_simulator = FdaSimulator()
domain_range = fda_simulator.domain_range
abscissa_points = fda_simulator.abscissa_points

In [ ]:
# Prepare some global parameters used to generate data
cnt = 30
alpha_p =  1 * cnt
beta_p = 3 * cnt
positions = [0.15, 0.35, 0.55, 0.85]
n_basis_simulated_data = 31

np.random.seed(1234)
X_cov, phi_X, epsilon, beta_data, col_indexes_bct, target_gen = fda_simulator.simulate(
    type_covariate="brownian_trend",
    type_transformation="linear_unimodal",
    sample_size=200,
    eta=0.05,
    datasets_type = ["train"],
    # Series representation
    n_basis_simulated_data=n_basis_simulated_data,
    sd_x=0.01,
    # Beta parameters
    alpha_param=alpha_p,
    beta_param=beta_p,
    # Brownian parameters
    intercept_brownian=0,
    slope_brownian=1,
    positions=positions
)

# Analyse mRMR

In [ ]:
X = X_cov[0]
target = target_gen[0]
domain_range = fda_simulator.domain_range
verbose = False
num_permutations = 1
predict_fns = None
labels_fns = None
num_intervals = 20
intervals = None
seed = None
compute_mrmr_distance_correlation = True
ini_domain_range = domain_range[0]
end_domain_range = domain_range[1]
long_domain_range = end_domain_range - ini_domain_range

In [ ]:
X[:4, :4]

In [ ]:
s = plt.plot(X.T)

In [ ]:
col_mean = np.mean(X, axis=0)
col_std = np.std(X, axis=0)
X_c =np.empty(shape=X.shape)
n_col = X.shape[1]
for i_col in range(n_col):
    X_c[:, i_col] = (X[:, i_col] - col_mean[i_col])/col_std[i_col]

In [ ]:
s = plt.plot(X_c[:20, :20].T)

In [ ]:
s = plt.plot(X_c[:20, 80:100].T)

In [ ]:
intervals_lower_bound = np.array(
    [ini_domain_range + i * long_domain_range/num_intervals for i in range(num_intervals)]
)
intervals_upper_bound = np.array(
    [ini_domain_range + (i + 1) * long_domain_range/num_intervals for i in range(num_intervals)]
)
intervals = np.stack((intervals_lower_bound, intervals_upper_bound), axis=1)
set_intervals = intervals.copy()
set_intervals_shape = set_intervals.shape
map_object = np.full(shape=abscissa_points.shape, fill_value=1, dtype=int)
num_intervals = set_intervals_shape[0]
last_end_interval = set_intervals[num_intervals-1, 1]
i_abscissa = 0
for abscissa in abscissa_points:
    if(np.abs(abscissa - last_end_interval) < 1e-7):
        interval_position = num_intervals - 1
    else:
        interval_position = np.ravel(np.argwhere((abscissa >= set_intervals[:, 0]) & (abscissa < set_intervals[:, 1])))
        interval_position = interval_position[0]
    map_object[i_abscissa] = interval_position
    i_abscissa += 1
middle_points = [(x[0] + x[1])/2 for x in intervals]

In [ ]:
m_list = []
redd_list = []
rell_list = []
for i_num_interval in range(num_intervals):
    desired_intervals = [i_num_interval]
    x_data = []
    num_cols = 0
    for i_interval in desired_intervals:
        position = np.argwhere(map_object == i_interval)
        f_i_interval = np.squeeze(X_c[:, position])
        #f_i_interval = np.squeeze(X[:, position])
        num_cols = num_cols + f_i_interval.shape[1]
        x_data.append(f_i_interval)
    f_available_abscissa = np.column_stack(x_data)
    corr_cef = np.corrcoef(f_available_abscissa, rowvar=False)
    redundancy = np.mean(np.abs(corr_cef))
    relevance_matrix = np.corrcoef(f_available_abscissa, target, rowvar=False)
    relevance = np.mean(np.abs(relevance_matrix[:, -1]))
    metric = relevance/redundancy
    m_list.append(metric)
    redd_list.append(redundancy)
    rell_list.append(relevance)


In [ ]:
fig, ax =  plt.subplots(1)
plt.plot(middle_points, redd_list, "o")
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.set_xlabel(r"$t$", fontsize=18)
ax.set_ylabel(r"$\text{Red}$", fontsize=18)
plt.savefig(os.path.join(images_path, "redundancy_brownian.pdf"), format="pdf", bbox_inches="tight")

In [ ]:
plt.plot(m_list)

In [ ]:
final_list = []
relevance_list = []
redundancy_list = []

for num_1 in range(num_intervals):
    metric_list = []
    rel_list = []
    red_list = []
    for num_2 in range(num_intervals):
        if num_1 != num_2:
            desired_intervals = [num_1, num_2]
            x_data = []
            num_cols = 0
            for i_interval in desired_intervals:
                position = np.argwhere(map_object == i_interval)
                f_i_interval = np.squeeze(X[:, position])
                num_cols = num_cols + f_i_interval.shape[1]
                x_data.append(f_i_interval)
            f_available_abscissa = np.column_stack(x_data)
            corr_cef = np.corrcoef(f_available_abscissa, rowvar=False)
            redundancy = np.mean(np.abs(corr_cef))
            relevance_matrix = np.corrcoef(f_available_abscissa, target, rowvar=False)
            relevance = np.mean(np.abs(relevance_matrix[:-1, -1]))
            metric = relevance/redundancy
            metric_list.append(metric)
            rel_list.append(relevance)
            red_list.append(redundancy)
    final_list.append(np.mean(metric_list))
    relevance_list.append(np.mean(rel_list))
    redundancy_list.append(np.mean(red_list))

In [ ]:
plt.plot(relevance_list)

In [ ]:
plt.plot(redundancy_list)

In [ ]:
plt.plot(final_list)

In [ ]:
print(rell_list)
print(relevance_list)

print(redd_list)
print(redundancy_list)

print(m_list)
print(final_list)

In [ ]:
#x_min = [x[0] for x in intervals]
#x_max = [x[1] for x in intervals]
#plt.hlines(
#    metric_list,
#    x_min,
#    x_max,
#    linewidth=3,
#)
#plt.step(
#    x_min,
#    metric_list,
#    where="post",
#    linestyle=(0, (1, 5)),
#)